Loading datasets

In [ ]:
# to be completed
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
%cd /content/drive/'My Drive'/'IEMOCAP_full_release_withoutVideos_sentenceOnly'/'IEMOCAP_full_release'

/content/drive/.shortcut-targets-by-id/1H_wIpsIF42ILnS8ZNz7BovI5ga272ikl/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release


In [ ]:
!pwd

Importing libraries

In [ ]:
!pip install tensorboardX

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

Preprocessing the data

In [ ]:
import numpy as np
import os
import pandas
import librosa

In [ ]:
def get_labels(annot_file, file_name):

    f = open(annot_file, 'r').read()
    f = f.split('\n')
    f = f[2:]

    for data in f:

        if len(data) > 0:
            if data[0] == '[':
                data2 = data.split('\t')

                if data2[1] == file_name:
                    emo = data2[2]
                    vad = data2[3][1:-1].split(', ')
                    return emo, [float(x) for x in vad]

    raise ValueError('Label not found')

def get_mocap_rot(path):

    f = open(path, 'r').read()
    f = np.array(f.split('\n'))
    header = f[0].split(' ')
    xyz = f[1].split(' ')
    f = f[2:]

    data_list = []

    for data in f:
        data2 = data.split(' ')
        if(len(data2)<2):
            continue
        dic = {'frame': data2[0], 'time': data2[1],
               'markers': np.array(data2[2:]).astype(float)}
        data_list.append(dic)

    return header, xyz, data_list

def get_wav(path):
    x, sr = librosa.load(path, sr=16000)
    return x, sr

def get_ph_fa(path):
    f = open(path, 'r').read()
    f = np.array(f.split('\n'))
    header = f[0].split()
    f = f[1:-2]
    data_list = []

    for data in f:
        data2 = data.split()
        dic = {'SFrm':np.array(data2[0]).astype(int),
               'EFrm':np.array(data2[1]).astype(int),
               'SegAScr':np.array(data2[2]).astype(int),
               'Phone':data2[3]}
        data_list.append(dic)

    return header, data_list

def frame_to_s(fr):
    return (fr+2)*10/1000

In [ ]:
root_path = '/content/gdrive/MyDrive/IEMOCAP_full_release_withoutVideos_sentenceOnly'

df = pandas.read_csv(os.path.join(root_path, 'iemocap.csv'))
sessions = [1, 2, 3, 4, 5]
df = df[df['session'].isin(sessions)]

# Remove unwanted emotions and empty values
unwanted_emotions = ['xxx', '', 'oth', 'dis', 'sur', 'fea']
df = df[~df['emotion'].isin(unwanted_emotions)]

# Calculate annotator difference
df['annotator_difference'] = df['n_annotators'] - df['agreement']

# Filter by annotator difference
df = df[df['annotator_difference'] <= 1]

# Replace 'exc' emotion with 'hap'
df.loc[df['emotion'] == 'exc', 'emotion'] = 'hap'


emotions_count_before = df['emotion'].value_counts()
print("Emotions count before filtering:")
print(emotions_count_before)

# Group by emotion and select first 550 rows of each group
df = df.groupby('emotion').head(550)

# Count the occurrences of each emotion after filtering
emotions_count_after = df['emotion'].value_counts()
print("\nEmotions count after filtering:")
print(emotions_count_after)

# Display the first 5 rows
display(df)
#Ntotal = 5531 utterances that are labeled with one of
#five categories: Happy, Angry, Neutral, Sad, Exited. This set is
#mapped down to four categories by merging Exited and Happy


Emotions count before filtering:
fru    1692
neu    1551
hap    1471
ang    1077
sad    1000
Name: emotion, dtype: int64

Emotions count after filtering:
neu    550
fru    550
ang    550
hap    550
sad    550
Name: emotion, dtype: int64


,Unnamed: 0,session,method,gender,n_annotators,agreement,emotion,valence,activation,dominance,wav_path,MOCAP_rotated_path,MOCAP_head_path,MOCAP_hand_path,FA_ph_path,FA_st_path,FA_sy_path,FA_wd_path,annotator_difference
0,0,1,script,F,3,3,neu,2.5,2.0,2.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,0
1,1,1,script,F,3,2,fru,2.5,2.0,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
4,4,1,script,F,3,2,neu,2.0,3.0,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
6,6,1,script,F,3,2,ang,2.0,3.5,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
7,7,1,script,F,3,2,ang,2.0,3.5,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,5350,3,impro,M,4,3,ang,2.0,3.0,4.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,1
5493,5493,3,script,M,3,3,ang,2.0,4.0,4.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,0
5494,5494,3,script,M,3,2,ang,3.0,2.5,3.0,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,IEMOCAP_full_release/Session3/sentences/Forced...,1
5497,5497,3,script,M,3,2,ang,2.0,4.0,3.5,IEMOCAP_full_release/Session3/sentences/wav/Se...,IEMOCAP_full_release/Session3/sentences/MOCAP_...,IEMOCAP_full_release/Session3/sentences/MOCAP_.

In [ ]:
for index, row in df.iterrows():

    session = row['session']
    method = row['method']
    gender = row['gender']
    emotion = row['emotion']
    n_annot = row['emotion']
    agreement = row['agreement']
    wav_path = row['wav_path']

    _, file_name = os.path.split(wav_path)

    annot_file = os.path.join(root_path, 'IEMOCAP_full_release',
                              'Session'+str(session),
                              'dialog', 'EmoEvaluation',
                              file_name[:-9] + '.txt')

    # get labels
    emo, vad = get_labels(annot_file, file_name[:-4])

    # get MOCAP data
    header_rot, xyz_rot, data_rot = get_mocap_rot(os.path.join(root_path, row['MOCAP_rotated_path']))
    print(xyz_rot)
    print(header_rot)
    print(data_rot)
    # get audio data
    x, sr = get_wav(os.path.join(root_path, row['wav_path']))

    # get transcription data
    header_ph, data_ph = get_ph_fa(os.path.join(root_path, row['FA_ph_path']))

    break

print(get_labels(annot_file, file_name[:-4]))
print(vad)
print(frame_to_s(data_ph[-1]['EFrm']))
print(x.shape[0]/sr)
print(data_rot[-1]['time'])


NameError: name 'df' is not defined

Preparing training / testing /  eval datasets

In this section we will use the markers

Defining the three models used to train MocapHead/MocapFace/MocapHand  
Inspired from this paper : https://www.researchgate.net/publication/324558351_Multi-Modal_Emotion_recognition_on_IEMOCAP_Dataset_using_Deep_Learning


First Model for MocapHead / MocapHand

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=256, output_size=4):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        x = hn[-1, :, :]
        x = self.fc(x)
        x = self.relu(x)
        return x


Second Model for MocapHead/MocapHand

In [ ]:
class DenseModel(nn.Module):
    def __init__(self, input_size, hidden_size=256, output_size=4):
        super(DenseModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


Models for MocapFace

First Model

In [ ]:
class FacialExprLSTM(nn.Module):
    def __init__(self, input_size, output_size=4):
        super(FacialExprLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, 512, batch_first=True)
        self.lstm2 = nn.LSTM(512, 256, batch_first=True)
        self.fc = nn.Linear(256, output_size)

    def forward(self, x):
        _, (ht1, _) = self.lstm1(x)
        _, (ht2, _) = self.lstm2(ht1)
        out = F.relu(self.fc(ht2[-1]))
        return out


Second Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model2(nn.Module):
    def __init__(self, input_channels=1, num_classes=4):
        super(Model2, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=2)
        self.dropout = nn.Dropout2d(0.2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.dropout(x)
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)



Third Model

In [ ]:
class Model3(nn.Module):
    def __init__(self, input_channels=1, num_classes=4):
        super(Model3, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 1024)  # Replace last convolution with a dense layer
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout2d(0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)
